In [1]:
from config import *
import utilities as utils

import importlib

# importlib.reload(utils)

year_files_dict = utils.get_all_filenames()

In [2]:
import gensim
import nltk
import pandas as pd
import re
import itertools

/user/ms5941/.local/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
stopwords_list = nltk.corpus.stopwords.words('english')

In [4]:
import json

### Get all tokenized sentences

In [ ]:
all_words = list()
tokenized_doc = dict()

for fname in itertools.chain.from_iterable(year_files_dict.values()):
    tokenized_sentences = list()
    data = utils.load_data(fname)
    count = 0
    for art in data['Items']:
        text = ''
        if art['data']['language'] == 'en':
            if any(subject in art['data']['subjects'] for subject in [US_CODE]):
                text = art['data']['body']

        if not text:
            continue
        count += 1

        text = nltk.sent_tokenize(text)
        text = [sentence.replace('\n', ' ') for sentence in text]
        doc_words = []
        for sentence in text:
            tokenized_sent = list()
            # Only keeping alphabets and replacing everything else with spaces
            sentence = re.sub(r'[^A-Za-z]+', ' ', sentence)
            sentence = sentence.lower()
            sent_words = nltk.word_tokenize(sentence)
            sent_words = [word for word in sent_words if ((len(word) > 1) and (len(word) < 20))]
            sent_words = [word for word in sent_words if word not in stopwords_list]
            for word in sent_words:
                doc_words.append(word)
                all_words.append(word)
                tokenized_sent.append(word)
            tokenized_sentences.append(tokenized_sent)

        tokenized_doc[art['data']['id']] = doc_words
    print(fname, 'Done! Article Count:', count, 'Storing in a json now')
    with open(TEMP_PATH + '/%s.txt' % fname[-15:-9], 'w') as outfile:
        json.dump(tokenized_sentences, outfile)
    del data

In [ ]:
del tokenized_doc

In [ ]:
len(all_words)

In [5]:
import glob
files = glob.glob(TEMP_PATH + '/2018*')

In [32]:
tokenized_sentences = []
for f in files:
    f = open(f)
    lines = f.readlines()
    tokenized_sentences.extend(eval(lines[0]))
    print(f, 'done!')

<_io.TextIOWrapper name='/work/ms5941/NLP/Temp/201808.txt' mode='r' encoding='UTF-8'> done!
<_io.TextIOWrapper name='/work/ms5941/NLP/Temp/201802.txt' mode='r' encoding='UTF-8'> done!
<_io.TextIOWrapper name='/work/ms5941/NLP/Temp/201803.txt' mode='r' encoding='UTF-8'> done!
<_io.TextIOWrapper name='/work/ms5941/NLP/Temp/201804.txt' mode='r' encoding='UTF-8'> done!
<_io.TextIOWrapper name='/work/ms5941/NLP/Temp/201801.txt' mode='r' encoding='UTF-8'> done!
<_io.TextIOWrapper name='/work/ms5941/NLP/Temp/201809.txt' mode='r' encoding='UTF-8'> done!
<_io.TextIOWrapper name='/work/ms5941/NLP/Temp/201811.txt' mode='r' encoding='UTF-8'> done!
<_io.TextIOWrapper name='/work/ms5941/NLP/Temp/201810.txt' mode='r' encoding='UTF-8'> done!
<_io.TextIOWrapper name='/work/ms5941/NLP/Temp/201807.txt' mode='r' encoding='UTF-8'> done!
<_io.TextIOWrapper name='/work/ms5941/NLP/Temp/201812.txt' mode='r' encoding='UTF-8'> done!
<_io.TextIOWrapper name='/work/ms5941/NLP/Temp/201805.txt' mode='r' encoding='UT

### Gensim Model - To get the similar words to our three themes

In [14]:
from gensim.models import Word2Vec

In [17]:
Word2Vec?

Init signature:
Word2Vec(
    sentences=None,
    corpus_file=None,
    vector_size=100,
    alpha=0.025,
    window=5,
    min_count=5,
    max_vocab_size=None,
    sample=0.001,
    seed=1,
    workers=3,
    min_alpha=0.0001,
    sg=0,
    hs=0,
    negative=5,
    ns_exponent=0.75,
    cbow_mean=1,
    hashfxn=<built-in function hash>,
    epochs=5,
    null_word=0,
    trim_rule=None,
    sorted_vocab=1,
    batch_words=10000,
    compute_loss=False,
    callbacks=(),
    comment=None,
    max_final_vocab=None,
)
Docstring:     
Serialize/deserialize objects from disk, by equipping them with the `save()` / `load()` methods.

Warnings
--------
This uses pickle internally (among other techniques), so objects must not contain unpicklable attributes
such as lambda functions etc.
Init docstring:
Train, use and evaluate neural networks described in https://code.google.com/p/word2vec/.

Once you're finished training a model (=no more updates, only querying)
store and use only the :class:

In [33]:
%env PYTHONHASHSEED = 1
model = Word2Vec(tokenized_sentences)
model.save("all_reuters_news_articles_2018.w2v")

env: PYTHONHASHSEED=1


In [34]:
pd.DataFrame(model.wv.most_similar('volatile'), columns=['Word', 'Similarity'])

,Word,Similarity
0,choppy,0.662459
1,muted,0.661966
2,weakness,0.604124
3,cyclical,0.600422
4,subdued,0.599536
5,broader,0.590146
6,steadier,0.587382
7,weaker,0.584046
8,compressed,0.574726
9,sideways,0.574022


In [35]:
pd.DataFrame(model.wv.most_similar('risk'), columns=['Word', 'Similarity'])

,Word,Similarity
0,risks,0.677940
1,likelihood,0.551028
2,possibility,0.533136
3,susceptible,0.530028
4,volatility,0.529373
5,pressure,0.518993
6,vulnerable,0.517497
7,uncertainties,0.514634
8,threat,0.506549
9,exposures,0.496108


In [36]:
model.wv.most_similar('volatility')

[('correlation', 0.6180945634841919),
 ('gyrations', 0.5870539546012878),
 ('skew', 0.5743195414543152),
 ('volatile', 0.5628951787948608),
 ('bullishness', 0.5513283610343933),
 ('vix', 0.5509589314460754),
 ('pullback', 0.5426290035247803),
 ('correction', 0.5406435132026672),
 ('fluctuations', 0.5396336317062378),
 ('premia', 0.5305648446083069)]

In [37]:
model.wv.most_similar('inflation')

[('cpi', 0.7215242385864258),
 ('unemployment', 0.5976287722587585),
 ('ppi', 0.5819039344787598),
 ('gdp', 0.5798148512840271),
 ('wages', 0.5700329542160034),
 ('hike', 0.5614430904388428),
 ('inflationary', 0.5343915224075317),
 ('hawkish', 0.5265888571739197),
 ('pmi', 0.524986743927002),
 ('hiking', 0.5240602493286133)]

In [40]:
model.wv.most_similar('bullishness')

[('nervousness', 0.6953359246253967),
 ('jitters', 0.6165590286254883),
 ('pessimism', 0.6074214577674866),
 ('uncertainty', 0.6003376245498657),
 ('angst', 0.5906758904457092),
 ('unease', 0.5842797160148621),
 ('uncertainties', 0.5819010734558105),
 ('valuations', 0.5816577672958374),
 ('skittish', 0.5803642868995667),
 ('anxiety', 0.580199658870697)]

In [38]:
model.wv.most_similar('gdp')

[('cpi', 0.6379778385162354),
 ('annualised', 0.5841681957244873),
 ('inflation', 0.5798148512840271),
 ('ppi', 0.552089512348175),
 ('nlnsugeedo', 0.5514264106750488),
 ('deficit', 0.5479311943054199),
 ('wage', 0.5265582799911499),
 ('pmi', 0.5194874405860901),
 ('eurozone', 0.5057522058486938),
 ('neoni', 0.5055637955665588)]

In [ ]:
# TODO
# IF we are going down this path, parallel processing on the grid is probably the best option.
# Or increasing memory allocation for notebook might help too. 